# Topic Modelling and Text Categorisation

In [1]:
import csv

import pandas as pd

from datasets import load_dataset

In [11]:
topic_data_raw = load_dataset("ag_news", trust_remote_code=True, split="train")
topic_data_raw

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [12]:
count = 0

for i in topic_data_raw:
    if count < 50:
        print(i)
    else:
        break

    count += 1

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'label': 2}
{'text': "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'label': 2}
{'text': 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'lab